In [1]:
import numpy as np
import pandas as pd

In [2]:
from classes.model import Model
from classes.scenario import Scenario
from classes.setup import Setup
from classes.combine import Combine

setup = Setup({
    'dir_output_all_runs': 'birds',  # Optional folder name
})


# SCENARIO 1


# Scenario overwrites default values
scenario = Scenario({
    'name': 'drip-and-ship',
    "setup": setup,
    "run_duration": 5,  # years
})

df_areas = scenario.get_model_areas()
row_mask = df_areas['isdn'].isin([
    'Devon, Cornwall and Isles of Scilly',
    'Gloucester, BSW, BNSSG and Somerset'
])
df_areas.loc[row_mask, 'selected'] = 1
scenario.set_model_areas(df_areas)

scenario.process_scenario()

model = Model(scenario)
model.run()

# SCENARIO 2

# Scenario overwrites default values
scenario = Scenario({
    'name': 'mothership',
    "setup": setup,
    "run_duration": 5,  # years
})


df_areas = scenario.get_model_areas()
row_mask = df_areas['isdn'].isin([
    'Devon, Cornwall and Isles of Scilly',
    'Gloucester, BSW, BNSSG and Somerset'
])
df_areas.loc[row_mask, 'selected'] = 1
scenario.set_model_areas(df_areas)

df_units = scenario.get_unit_services()
# Only keep services at MT units.
row_mask = df_units['use_mt'] == 0
df_units.loc[row_mask, ['use_ivt', 'use_mt', 'use_msu', 'selected']] = 0

scenario.set_unit_services(df_units)

scenario.process_scenario()

# scenario.set_lsoa_catchment_type('nearest')
# scenario._create_lsoa_travel_dicts()
# scenario._load_admissions()

model = Model(scenario)

model.run()

# Combine results across all runs
combine = Combine({'setup': setup})
combine.combine_files()

setup.save_to_file()

## Testing

In [3]:
print(stop, here, please)

NameError: name 'stop' is not defined

In [ ]:
import os
# Merge in transfer unit names.
# Load and parse hospital transfer data
dir_input = setup.dir_output
file_input = setup.file_national_transfer_units
path_to_file = os.path.join(dir_input, file_input)
transfer = pd.read_csv(path_to_file)
# transfer = transfer.rename(columns={'from_postcode': 'Postcode'})
transfer = transfer.drop(['time_nearest_mt'], axis='columns')
# Index: 'Postcode'
# Columns: 'name_nearest_mt'

dir_input = setup.dir_output
file_input = setup.file_selected_stroke_units
path_to_file = os.path.join(dir_input, file_input)
hospitals = pd.read_csv(path_to_file)
# Index: 'Postcode'
# Columns: names, services, regions etc. ...
# Limit to selected stroke units.
# hospitals = hospitals[hospitals['selected'] == 1]

# Keep a copy of the coordinates:
hospital_coords = hospitals.copy()
hospital_coords = hospital_coords[[
    'Postcode', 'Easting', 'Northing', 'long_x', 'lat_x']]  # TO DO - Fix this annoying suffix


In [ ]:
transfer

In [ ]:
hospital_coords

In [ ]:
m1 = pd.merge(
    transfer, hospital_coords,
    left_on='from_postcode', right_on='Postcode',
    how='right'
    )
m1
# m1 = m1.drop(['Postcode_mt', 'Postcode'], axis='columns')


In [ ]:
m2 = pd.merge(
    m1, hospital_coords,
    left_on='name_nearest_mt', right_on='Postcode',
    how='left', suffixes=(None, '_mt')
    )
transfer_hospitals = m2.drop(['Postcode', 'Postcode_mt'], axis='columns')


# Limit to selected stroke units.
selected_units = hospitals['Postcode'][hospitals['selected'] == 1]
mask = transfer_hospitals['from_postcode'].isin(selected_units)
transfer_hospitals = transfer_hospitals[mask]

transfer_hospitals = transfer_hospitals.set_index('from_postcode')

transfer_hospitals

In [ ]:
# Merge in the arrival unit coordinates:
transfer_hospitals = pd.merge(
    hospital_coords, m1,
    left_on='Postcode', right_on='Postcode',
    how='left', suffixes=(None, '_mt')
)

In [ ]:

# Merge in the transfer unit coordinates:
transfer = pd.merge(
    transfer, hospital_coords,
    left_on='name_nearest_mt', right_on='Postcode',
    how='right', suffixes=('_mt', None)
    )
# Merge in the arrival unit coordinates:
transfer_hospitals = pd.merge(
    hospital_coords, transfer,
    left_on='Postcode', right_on='Postcode',
    how='left', suffixes=(None, '_mt')
)
transfer_hospitals = transfer_hospitals.set_index('Postcode')
# TO DO - tidy up the excess columns in hospitals ---------------------------


In [ ]:
df_units[df_units['selected'] == 1]

In [ ]:
print(stop, here, please)

In [ ]:
setup = Setup()
setup.import_from_file('./output/birds!18/setup.yml')

In [ ]:
vars(scenario)

In [ ]:
from classes.map import Map
plot_map_selected_units(
    setup, col=scenario.region_column_for_lsoa_selection)
# TEMPORARILY commented out for speed of running. 29th Jan 2024
plot_map_catchment(
    setup, col=scenario.region_column_for_lsoa_selection)
plot_map_outcome(
    setup, col=scenario.region_column_for_lsoa_selection,
    outcome='mrs_shift', destination_type=scenario.destination_decision_type
)


In [ ]:
completed_patients_keys = model.pathway.completed_patients[0].keys()
time_cols = [x for x in completed_patients_keys if x[0:4] == 'time']
time_cols.remove('time_onset')

In [ ]:
model.results_summary_all

In [ ]:
model.results_all

In [ ]:
model.results_summary_by_admitting_unit

In [ ]:
model.results_all[np.isnan(model.results_all['time_puncture']) == False]

In [ ]:
model.results_all[np.isnan(model.results_all['time_needle']) == False]